In [1]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c domain-network-security

In [3]:
!unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
pd.set_option("Max_Columns",999)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
from sklearn.metrics import f1_score
pd.set_option("Max_Columns",999)

In [75]:
df=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [76]:
df.head()

,ID,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received,Action
0,59276,49418.0,27015.0,54748.0,NaN,78.0,78.0,0.0,1.0,29.0,1.0,0.0,allow
1,18986,58120.0,25174.0,NaN,0.0,66.0,66.0,0.0,NaN,0.0,1.0,0.0,deny
2,11131,46751.0,53.0,46751.0,53.0,220.0,90.0,130.0,2.0,30.0,NaN,1.0,allow
3,57352,52596.0,NaN,0.0,0.0,66.0,66.0,0.0,1.0,NaN,1.0,0.0,drop
4,17229,50584.0,6881.0,29773.0,NaN,502.0,150.0,352.0,2.0,1200.0,NaN,1.0,allow


In [84]:
len(df)

45870

In [85]:
9187/45870

0.2002834096359276

In [77]:
df.isnull().sum()

ID                         0
Source Port             9187
Destination Port        9177
NAT Source Port         9114
NAT Destination Port    9196
Bytes                   9263
Bytes Sent              9106
Bytes Received          9177
Packets                 9052
Elapsed Time (sec)      9286
pkts_sent               9203
pkts_received           9159
Action                     0
dtype: int64

In [78]:
test.isnull().sum()

ID                         0
Source Port             3969
Destination Port        3979
NAT Source Port         4067
NAT Destination Port    3942
Bytes                   3877
Bytes Sent              3778
Bytes Received          3929
Packets                 3926
Elapsed Time (sec)      3959
pkts_sent               3954
pkts_received           3949
dtype: int64

In [82]:
len(test)

19662

In [86]:
4067/20000   # Similar Missing count. (Values maybe removed manually and split)

0.20335

In [80]:
df["Action"].value_counts(normalize=True)
#57 percent is allow 
# May be undersampling of allow 22 % . Reset both is very small( Have to oversample reset both or use smote  )
#no need to use smote as only 37 Samples are there

allow         0.574406
deny          0.228690
drop          0.196098
reset-both    0.000807
Name: Action, dtype: float64

In [34]:
for i in range(len(df)) :
        df.loc[i,"nancount"]= df.iloc[i].isnull().sum()

In [35]:
test["nancount"]=0
for i in range(len(test)) :
        test.loc[i,"nancount"]= test.iloc[i].isnull().sum()

In [ ]:
df["nancount"].value_counts()
#can drop rows with 7,8,9 nan values

In [36]:
df =df[df["nancount"]<7].copy()

In [37]:
df["nancount"].value_counts()

3.0    13583
2.0    10818
4.0    10163
5.0     5021
1.0     3937
6.0     1823
Name: nancount, dtype: int64

In [38]:
df[["Bytes","Bytes Sent","Bytes Received"]]

,Bytes,Bytes Sent,Bytes Received
0,78.0,78.0,0.0
1,66.0,66.0,0.0
2,220.0,90.0,130.0
3,66.0,66.0,0.0
4,502.0,150.0,352.0
...,...,...,...
45865,94.0,94.0,0.0
45866,177.0,94.0,83.0
45867,70.0,NaN,0.0
45868,2293.0,1576.0,NaN


### Found a Pattern Between Bytes ,Bytes _Sent and Recieved
### Packets ,Packets Sent and Packets Recieved


In [39]:
df.rename(columns={"Bytes Sent":"Bytes_Sent","Bytes Received":"Bytes_Recieved"},inplace=True)

In [40]:
df["Bytes_Sent"].fillna(df.Bytes -df.Bytes_Recieved,inplace=True)

In [43]:
df[["Bytes","Bytes_Sent","Bytes_Recieved"]]

,Bytes,Bytes_Sent,Bytes_Recieved
0,78.0,78.0,0.0
1,66.0,66.0,0.0
2,220.0,90.0,130.0
3,66.0,66.0,0.0
4,502.0,150.0,352.0
...,...,...,...
45865,94.0,94.0,0.0
45866,177.0,94.0,83.0
45867,70.0,70.0,0.0
45868,2293.0,1576.0,717.0


In [42]:
df["Bytes_Recieved"].fillna(df.Bytes -df.Bytes_Sent,inplace=True)

In [45]:
df["Packets"].fillna(df.pkts_sent + df.pkts_received,inplace=True)

In [46]:
df["pkts_sent"].fillna(df.Packets - df.pkts_received,inplace=True)

In [47]:
df["pkts_received"].fillna(df.Packets -df.pkts_sent ,inplace=True)

In [51]:
df.isnull().sum()

ID                         0
Source Port             8889
Destination Port        8862
NAT Source Port         8826
NAT Destination Port    8904
Bytes                   8965
Bytes_Sent              2978
Bytes_Recieved          3008
Packets                 3079
Elapsed Time (sec)      8985
pkts_sent               3079
pkts_received           3020
Action                     0
nancount                   0
dtype: int64

In [87]:
## Reduced Null Values  by three Fold in Bytes and Packets

## Undersampling of Allow and Oversampling of "reset Both"

In [64]:
allowdf=df[df["Action"]=="allow"].sample(n=15000,random_state=42)
reset_df=df[df["Action"]=="reset-both"].sample(n=2000,random_state=42,replace=True)

In [65]:
df=pd.concat([allowdf,reset_df,df[df["Action"]=="deny"],df[df["Action"]=="drop"]],axis=0)

In [66]:
df["Action"].value_counts()

allow         15000
deny          10368
drop           8906
reset-both     2000
Name: Action, dtype: int64

**LightGBM performed better than XGBoost and RandomForest in Cross Validation so using them**

In [67]:
from lightgbm import LGBMClassifier

In [68]:
lgb=LGBMClassifier(objective="multi:softprob",verbosity=1,n_estimators=200)
lgb.fit(df.drop("Action",axis=1),df["Action"])

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=200, n_jobs=-1, num_leaves=31,
               objective='multi:softprob', random_state=None, reg_alpha=0.0,
               reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0, verbosity=1)

In [69]:
pred=lgb.predict(test)

In [70]:
pd.Series(pred).value_counts()

allow         11281
deny           4478
drop           3898
reset-both        5
dtype: int64

In [72]:
lgbundersampled=pd.DataFrame({"ID":test["ID"],"Action":pred})
lgbundersampled.to_csv("lgbundersampled.csv",index=False)